<a href="https://colab.research.google.com/github/jiahfong/incoherent-thoughts/blob/develop/BatchBALD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BatchBALD

# Introduction

BALD is useful in the setting where only one sample is acquired at a time; however, when multiple samples are acquired at once, BALD would select many similar points as it has no "memory" that it had acquired previous points (assuming the usual method where the top-b scoring BALD points are taken). BatchBALD considers the _joint_ mutual information of all the points in the batch. Further, because it is _submodular_, it is at worst a $1 - \frac{1}{\epsilon}$ approximation of the optimal choice when a greedy algorithm is used to choose the batch (to avoid exponential search). Hence, this paper's contributions is threefold:

1. derivation of BatchBALD.
2. a greedy algorithm to select the batch of $b$ points effeciently.
3. estimator for BatchBALD that scales to larger acquisition sizes and datasets with many classes.

# Background

## BALD
BALD estimates the mutual information (MI) between the model's predictions on the unlabelled pool and the model's parameters. Finding out the true label of the model's predictions with high MI should inform us of the true model parameters:

$$
\mathbb{I}[y; \mathbf{\omega}\ |\ \mathbf x, \mathcal{D}_{train}] = \mathbb{H}(y\ |\ \mathbf x, \mathcal{D}_{train}) - \mathbb{E}_{p(\mathbf \omega|\mathcal{D}_{train})}[\mathbb{H}(y\ |\ \mathbf x, \mathbf \omega, \mathcal{D}_{train})]
$$

To maximise mutual information, the left term must be high (implies that the model is uncertain in its predictions over all possible parameters $\omega$) and the right term must be low (implies that the model is very certain in its predictions for each _individual_ realisation of parameters $\omega$). Consider the extreme case where given $k$ realisations of $\omega$ and $k$ classes, each of the $k$ predictions have softmax probability of 1 at different classes (e.g. a diagonal matrix of shape $k$x$k$).

> Not suitable for deep learning as acquiring one point at a time is terribly inefficient and costly.

## Bayesian neural networks

Performing exact inference in BNNs is intractable for any non-trivial model; MC dropout is used as variational approximation.

# Methods

## BatchBALD

In essence, to avoid double counting, BatchBALD estimates the mutual information of _joint data points_ and the model parameters (omitting conditioning on $x_1, \dots, x_n$ and $\mathcal{D}_{train}$ in both terms on RHS and $p(\omega)$ denotes $p(\omega|\mathcal{D}_{train}))$:

$$
\mathbb{I}[y_1, y_2, \dots, y_n; \mathbf{\omega}\ |\ x_1, x_2, \dots, x_n, \mathcal{D}_{train}] = \mathbb{H}(y_1, \dots, y_n) - \mathbb{E}_{p(\omega)}[\mathbb{H}(y_1, \dots, y_n | \omega)]
$$

where:

$$
\begin{align}
\mathbb{E}_{p(\omega)}[\mathbb{H}(y_1, \dots, y_n | \omega)] &= \displaystyle\sum_{i=1}^{n}\mathbb{E}_{p(\omega)}[\mathbb{H}(y_i|\omega)] \ \ \ \text{($y_i$s are independent conditioned on $\omega$)} \\
&\approx \frac{1}{k}\displaystyle\sum_{i=1}^{n}\sum_{j=1}^{k}\mathbb{H}(y_i|\hat \omega_j)\ \ \text{($k$ monte-carlo estimates $\hat \omega_j \sim p(\omega)$)}
\end{align}
$$

and 

$$
\begin{align}
\mathbb{H}(y_1, \dots, y_n) &= -\displaystyle\sum_{y_1}\dots\sum_{y_n}p(y_1, \dots, y_n)log\ p(y_1, \dots, y_n) \\
&\approx  -\displaystyle\sum_{y_1}\dots\sum_{y_n}\left(\frac{1}{k}\sum_{j=1}^{k}p(y_1, \dots, y_n | \hat \omega_j)\right)log\ \left(\frac{1}{k}\sum_{j=1}^{k}p(y_1, \dots, y_n | \hat \omega_j)\right) \ \ \text{(1)}
\end{align}
$$

where in (1), $p(y_1, \dots, y_n) = \int p(y_1, \dots, y_n | \omega)\ p(\omega)\ d\omega$ is approximated using monte-carlo estimate.


## Greedy algorithm

The pseudocode looks like:

0. `acquired_points` <- $\varnothing$
1. for n = 1 .. b:
2. `acquired_points` <- `acquired_points` $\cup$ {$argmax_{ x \in \mathcal{D}_{pool} \text{\ acquired_points}}$ `BatchBALD`(`acquired_points` $\cup$ {$x$})} 


## Efficient implementation

In the above greedy algorithm, notice that $x_1, \dots, x_{n-1}$ (`acquired_points` before step 2) are always fixed when greedily searching for $x_n$. Define:

$$
\begin{align}
\left(\hat P_{1:n-1}\right)_{i, j} &= p(y_1^{(i)}, \dots, y_{n-1}^{(i)} | \hat \omega_j) \ \ &\text{(matrix of shape $\mathbb{R}^{c^{n-1}\times k}$)} \\ 
\left(\hat P_{n}\right)_{i, j} &= p(y_n^{(i)} | \hat \omega_j) \ \ &\text{(matrix of shape $\mathbb{R}^{c\times k}$)} \\ 
\frac{1}{k}\displaystyle\sum_{j=1}^{k}p(y_1, \dots, y_n | \hat \omega_j) &= \frac{1}{k} \displaystyle \sum_{j=1}^{k} p(y_1, \dots, y_{n-1} | \hat \omega_j)p(y_n | \hat \omega_j) = \left(\frac{1}{k}\hat P_{1:n-1}\hat P^{T}_{n}\right)_{y_{1:n-1}, y_n}\ \ &\text{(matrix of shape $\mathbb{R}^{c^{n-1}\times c}$)}
\end{align}
$$

where $p(y_1^{(i)}, \dots, y_{n-1}^{(i)} | \hat \omega_j)$ denotes the $i^{th}$ permutation of the random vector $\mathbf y_{1:n-1}^{(i)}$ (total of $c^{n-1}$ possible permutations) and the $j^{th}$ MC dropout sample.

Therefore, in each iteration of the greedy algorithm, $\hat P_{1:n-1}$ only needs to be computed once and the joint entropy for the $n^{th}$ addition, $x_n \in \mathcal{D}_{pool}$ \ `acquired_points`, can be calculated using batch matrix multiplication by concatenating mutiple candidate $x_n$s to the rows of $\hat P_n$.

### The problem, and the solution

#### Problem
However, $c^{n-1}$ is quite memory-demanding as a reasonable dataset with 10 classes and batch size 10 has 10 giga-rows.

#### Solution
Proposal to overcome intractable $c^{n-1}$ configurations using sampling.

TODO: see appendix on using $m$ rows instead of $c^{n-1}$ to approximate

TODO: assuming $p(y_1, \dots, y_n | \hat \omega_j)$ factorises to $\prod_i^n p(y_i | \hat \omega_j)$, wouldn't the product of probabilities underflow for large $n$?

# Experiments

## Practical considerations
* Labelled dataset is usually small in the beginning (e.g. 20 points); it is important to avoid overfitting. The authors used _early stopping after 3 epochs of declining accuracy on the validation set_.
* The models are reinitialised after each acquisition rounds. Empirically, it helped the model improve even when very small batches are acquired & decorrelates subsequent acquisitions.
* _Dropout masks_ are kept consistent _within_ the same acqusition round. This reduces variance and is necessary to capture dependencies between the inputs ($x_1, \dots, x_{n-1}$) and makes the scores computed for different points ($x \in \mathcal{D}_{pool}$) more comparable.
* In the following experiments, the authors used either 10,000 MC samples ($m$ above) or directly computed the joint entropy by enumerating all $c^{n-1}$ configurations (they used MC samples when $b > 4$).


## Repeated MNIST
The MNIST dataset is replicated twice (yielding 3x the size) where the duplicates are normalised and isotropic gaussian noise ($\sigma = 0.1$) added to it.

BatchBALD had no problems overcoming the replicated dataset; BALD performs worse than random acquisition.

(acquisition batch size = 10; 10 MC dropout samples)

## MNIST

* Increasing batch acquisision size for BALD quickly deteriorates its performance whilst BatchBALD maintains its performance (only slightly worse when batch size is 40, the authors postulates that it's due to estimator noise)
* BALD with acquisition size 1 and BatchBALD with acquisition size 10 performs very similarly, unsurprisingly outperforming BALD with size 10. Further, the time needed to reach 95% accuracy using BALD with size 1 is significantly longer than size 10, of which BatchBALD is only slightly slower than BALD.

(varying acquisition batch size; 100 MC dropout samples)

## EMNIST

* BatchBALD > random acquisition > BALD
* Entropy (of categorical distribution of classes) of BatchBALD is much higher than BALD, implying a more diverse set of acquired points. (cf. ICAL's postulation about lower entropy when number of classes increases)

(__no initial dataset__, model randomly initialised when performing the initial acquisition step; acquisition batch size = 5; 10 MC dropout samples)

## CINIC-10

* BatchBALD > BALD

(pretrained ImageNet model; acquisition batch size = 10; 50 MC dropout samples)

## Comments

The authors mostly used acquisition batch size of less than 40 in all experiments while ICAL's authors used generous acquisition sizes of up to 3000 (CIFAR-10/100 experiments). They were able to train the models (for CIFAR-10) up to 80% accuracy that way while BatchBALD's authors trained theirs (CINIC-10) up to 60% accuracy because the final dataset size is significantly smaller (1,400 in CINIC-10 vs 35,000 in CIFAR-10). ICAL's authors mentioned that BatchBALD ran out of memory (not sure if they tried sampling the joint entropies instead of enumerating all configurations).

# Scopes & Limitations

1. __Noisy estimator__: As the experiments showed, larger acquisition sizes does deteriorate BatchBALD. The authors postulate that it might be caused by the sampling of the joint entropies (introduces noise) and a significant amount of noise introduced by MC-dropout's VI approximation.

2. __Unlabelled data__: Semi-supervised extension to fully leverage the unlabelled dataset

3. __Unbalanced dataset__: Both BALD and BatchBALD is expected to not perform well when the test set is unbalanced as they aim to learn well about all classes and do not follow the density of the dataset.

# Closing remarks (relatability to AL project)

TODO